# caso Practico : Random Forest
En este caso de uso practico se pretende resolver un problema de deteccion de malware en
dispositivos android mediante el analisis del trafico de red que genera el dispositivo 
medinte el uso de conjuntos de árboles de decisión 

### DataSet: Deteccion de Malware en Android

#### Descripcion
The sophisticated and advanced Android malware is able to identify the presence of the emulator used by the malware analyst and in response, alter its behavior to evade detection. To overcome this issue, we installed the Android applications on the real device and captured its network traffic. See our publicly available Android Sandbox.

CICAAGM dataset is captured by installing the Android apps on the real smartphones semi-automated. The dataset is generated from 1900 applications with the following three categories:

**1. Adware (250 apps)**
* Airpush: Designed to deliver unsolicited advertisements to the user’s systems for information stealing.
* Dowgin: Designed as an advertisement library that can also steal the user’s information.
* Kemoge: Designed to take over a user’s Android device. This adware is a hybrid of botnet and disguises itself as popular apps via repackaging.
* Mobidash: Designed to display ads and to compromise user’s personal information.
* Shuanet: Similar to Kemoge, Shuanet also is designed to take over a user’s device.

**2. General Malware (150 apps)**
* AVpass: Designed to be distributed in the guise of a Clock app.
* FakeAV: Designed as a scam that tricks user to purchase a full version of the software in order to re-mediate non-existing infections.
* FakeFlash/FakePlayer: Designed as a fake Flash app in order to direct users to a website (after successfully installed).
* GGtracker: Designed for SMS fraud (sends SMS messages to a premium-rate number) and information stealing.
* Penetho: Designed as a fake service (hacktool for Android devices that can be used to crack the WiFi password). The malware is also able to infect the user’s computer via infected email attachment, fake updates, external media and infected documents.

**3. Benign (1500 apps)**
* 2015 GooglePlay market (top free popular and top free new)
* 2016 GooglePlay market (top free popular and top free new)

### Ficheros de datos
* pcap files – the network traffic of both the malware and benign (20% malware and 80% benign)
* <span style="color:green">.csv files - the list of extracted network traffic features generated by the CIC-flowmeter</span>

### Descarga de los ficheros de datos
https://www.unb.ca/cic/datasets/android-adware.html

### Referencias adicionales sobre el conjunto de datos
_Arash Habibi Lashkari, Andi Fitriah A. Kadir, Hugo Gonzalez, Kenneth Fon Mbah and Ali A. Ghorbani, “Towards a Network-Based Framework for Android Malware Detection and Characterization”, In the proceeding of the 15th International Conference on Privacy, Security and Trust, PST, Calgary, Canada, 2017._

Imports

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score

## Fuciones Auxiliares

In [15]:
# construccion de una funcion que realice el particionado completo 
def train_val_test_split(df, rstate = 42, shuffle = True, stratify = None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size = 0.4, random_state =rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else  None
    val_set, test_set = train_test_split(
        test_set, test_size = 0.5, random_state = rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [16]:
def remove_labels(df, label_name):
    X = df.drop(label_name, axis=1)
    y = df[label_name].copy()
    return (X, y) 

In [17]:
def evaluate_resul(y_pred, y, y_prep_pred, y_prep, metric):
    print(metric.__name__, "WITHOUT preparation:", metric(y_pred, y, average='weighted'))
    print(metric.__name__, "WITH preparation:", metric(y_prep_pred, y_prep, average='weighted'))

### 1.- Lectura del DataSet

In [18]:
df = pd.read_csv('datasets/datasets/TotalFeatures-ISCXFlowMeter.csv')

### 2.- visualizacion del DataSet

In [19]:
df.head(10)

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward,calss
0,1020586,668,1641,35692,2276876,52,52,679,1390,53.431138,1387.492992,25.850592,54.220784,1012372,1004743,1,2,33003,37595,1517.79910,612.648171,3430.256494,1961.288204,3,16,0,0,13360,32820,654.525929,1607.899775,2262.425704,2.265922e+06,52,1390,1001.545258,606.929295,-30,20674,442.194974,1208.029126,0,2,0,19,2308,0,0,0,63.792334,1001.545258,53.431138,13360,376,7,238318,2939.893396,1387,13535,9,4961514,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,2,4194240,1853440,1640,668,32,benign
1,80794,1,1,75,124,75,124,75,124,75.000000,124.000000,0.000000,0.000000,0,0,-1,-1,-1,-1,0.00000,0.000000,0.000000,0.000000,0,0,0,0,20,20,12.377157,12.377157,24.754313,2.463054e+03,75,124,99.500000,34.633799,80794,80794,80794.000000,0.000000,0,0,0,0,0,0,0,0,1.653333,99.500000,75.000000,20,0,0,0,0.000000,124,0,0,0,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,2,0,0,0,1,0,benign
2,998,3,0,187,0,52,-1,83,-1,62.333333,0.000000,17.883885,0.000000,998,0,2,-1,996,-1,499.00000,0.000000,702.863429,0.000000,1,0,0,0,60,0,3006.012024,0.000000,3006.012024,1.873747e+05,52,83,62.333333,17.883885,2,996,499.000000,702.863429,0,0,1,1,3,0,0,0,0.000000,62.333333,62.333333,60,0,0,0,0.000000,0,0,0,0,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,4,101888,-1,0,3,32,benign
3,189868,9,9,1448,6200,52,52,706,1390,160.888889,688.888889,216.161944,665.690045,137870,161424,2,11,40746,71705,17233.75000,20178.000000,17780.639850,29411.675150,3,4,0,0,180,180,47.401353,47.401353,94.802705,4.028062e+04,52,1390,424.888889,551.654345,2,51972,11168.705880,17180.912650,1,2,0,7,17,0,0,0,4.281768,424.888889,160.888889,180,334,4,8196,443143.236100,688,4201,4,2879369,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,2,4194240,2722560,8,9,32,benign
4,110577,4,6,528,1422,52,52,331,1005,132.000000,237.000000,134.065904,377.398728,82246,81939,3,12,80371,49456,27415.33333,16387.800000,45870.472680,22885.047450,2,4,0,0,80,120,36.173888,54.260832,90.434720,1.763477e+04,52,1005,195.000000,296.747367,3,49456,12286.333330,18425.638810,0,0,0,6,10,0,0,0,2.693182,195.000000,132.000000,80,0,0,0,142429.800000,237,1370,5,27150,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,2,155136,31232,5,4,32,benign
5,261876,7,6,1618,882,52,52,730,477,231.142857,147.000000,290.112581,170.474045,261876,186534,2,454,95625,77201,43646.00000,37306.800000,39724.781940,27948.162010,3,2,0,0,140,120,26.730208,22.911607,49.641815,9.546503e+03,52,730,192.307692,236.849771,2,72473,21823.000000,22332.506940,0,2,0,5,12,0,0,0,0.545117,192.307692,231.142857,140,0,0,0,29061.400000,147,0,0,0,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,2,4194240,926720,3,7,32,benign
6,14,2,0,104,0,52,-1,52,-1,52.000000,0.000000,1.000000,0.000000,14,0,14,-1,14,-1,14.00000,0.000000,0.000000,0.000000,0,0,0,0,40,0,142857.142900,0.000000,142857.142900,7.428571e+06,52,52,52.000000,1.000000,14,14,14.000000,0.000000,0,0,0,0,2,0,0,0,0.000000,52.000000,52.000000,40,0,0,0,0.000000,0,0,0,0,0,0,0,0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,3,5824,-1,0,2,32,benign
7,29675,1,1,71,213,71,213,71,213,71.000000,213.000000,0.000000,0.000000,0,0,-1,-1,-1,-1,0.00000,0.000000,0.000000,0.0

In [20]:
df.describe()

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward
count,6.319550e+05,631955.000000,631955.000000,6.319550e+05,6.319550e+05,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,631955.000000,631955.000000,631955.0,631955.0,631955.000000,6.319550e+05,6.319550e+05,631955.000000,6.319550e+05,6.319550e+05,631955.000000,631955.000000,631955.000000,631955.000000,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,631955.000000,631955.000000,631955.00000,631955.00000,631955.000000,631955.0,631955.0,631955.0,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,631955.000000,6.319550e+05,631955.000000,631955.000000,631955.000000,631955.000000,6.319550e+05,631955.000000,6.319550e+05,631955.000000,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,6.319550e+05,631955.000000,6.319550e+05,6.319550e+05,631955.000000,631955.00000,631955.000000
mean,2.195245e+07,6.728514,10.431934,9.540172e+02,1.206042e+04,141.475727,44.357688,263.675901,183.248084,174.959706,105.546055,53.014212,61.454557,1.633985e+07,7.132276e+06,1.170509e+07,9.375647e+05,1.530244e+07,6.284945e+06,1.275784e+07,2.047904e+06,1.769660e+06,2.240714e+06,0.858734,1.057927,0.0,0.0,134.654176,2.086387e+02,2.974404e+04,1731.437342,3.147547e+04,1.682912e+06,140.473502,340.888032,201.354935,84.865349,1.167877e+07,2.079031e+07,1.332030e+07,3.396481e+06,0.530555,0.429222,0.06706,1.91666,16.414992,0.0,0.0,0.0,1.243868,201.354935,174.959706,134.654176,52.822829,0.333408,2.284463e+05,28367.889759,105.447230,454.686911,0.491646,2.148029e+05,3.053119,4.221001e+02,4.886468,5.962097e+03,2.004092e+07,2.037212e+07,2.081310e+07,4.645655e+05,1.997327e+07,2.031228e+07,2.075238e+07,4.663875e+05,2.360896,9.620796e+05,3.104519e+05,9.733144,6.72471,19.965713
std,1.900578e+08,174.161354,349.424019,8.235040e+04,4.824716e+05,157.680880,89.099554,289.644383,371.863224,162.024811,206.667634,122.899076,156.816045,1.798612e+08,6.332736e+07,1.607115e+08,4.081212e+07,1.797718e+08,6.291667e+07,1.654258e+08,4.376971e+07,5.630759e+07,2.656013e+07,3.983274,13.346000,0.0,0.0,3484.817220,6.988480e+03,1.633361e+05,22444.835598,1.645742e+05,9.239142e+06,153.266293,383.732807,172.537279,159.429483,1.607078e+08,1.899679e+08,1.669399e+08,6.658424e+07,0.652985,0.911039,0.30462,16.44443,514.737161,0.0,0.0,0.0,5.013630,172.537279,162.024811,3484.817220,571.950886,3.274184,6.517195e+06,88098.474276,206.563178,2539.571052,2.146704,3.557865e+06,144.009491,6.655344e+04,265.053479,3.667979e+05,1.898094e+08,1.898005e+08,1.899846e+08,6.192833e+06,1.897986e+08,1.897902e+08,1.899721e+08,6.199704e+06,3.041810,1.705655e+06,6.647956e+05,347.877923,174.13813,14.914261
min,-1.800000e+01,0.000000,0.000000,0.000000e+00,0.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631955 entries, 0 to 631954
Data columns (total 80 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration                 631955 non-null  int64  
 1   total_fpackets           631955 non-null  int64  
 2   total_bpackets           631955 non-null  int64  
 3   total_fpktl              631955 non-null  int64  
 4   total_bpktl              631955 non-null  int64  
 5   min_fpktl                631955 non-null  int64  
 6   min_bpktl                631955 non-null  int64  
 7   max_fpktl                631955 non-null  int64  
 8   max_bpktl                631955 non-null  int64  
 9   mean_fpktl               631955 non-null  float64
 10  mean_bpktl               631955 non-null  float64
 11  std_fpktl                631955 non-null  float64
 12  std_bpktl                631955 non-null  float64
 13  total_fiat               631955 non-null  int64  
 14  tota

In [22]:
print("longitud del DataSet", len(df))
print("numerode columnas del DataSet", len(df.columns))

longitud del DataSet 631955
numerode columnas del DataSet 80


In [23]:
df['calss'].value_counts()

calss
benign            471597
asware            155613
GeneralMalware      4745
Name: count, dtype: int64

### Buscando correlaciones

In [24]:
# Trasformar la variable de salida a numerica para calcular correlaciones
X = df.copy()
X['calss'] = X['calss'].factorize()[0]
corr_matrix = X.corr()
corr_matrix['calss'].sort_values(ascending=False)

calss                     1.000000
flow_fin                  0.286175
min_seg_size_forward      0.258352
Init_Win_bytes_forward    0.129425
std_fpktl                 0.123758
                            ...   
furg_cnt                       NaN
burg_cnt                       NaN
flow_urg                       NaN
flow_cwr                       NaN
flow_ece                       NaN
Name: calss, Length: 80, dtype: float64

In [26]:
X.corr()

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward,calss
duration,1.000000,0.004837,0.004011,0.001673,0.003518,-0.064100,-0.027231,0.008761,0.042925,-0.043746,0.025117,0.039350,0.048743,0.943898,0.324705,0.841692,0.212482,0.943438,0.324226,0.918036,0.280637,0.421540,0.237307,0.021372,0.010780,NaN,NaN,0.004835,0.004011,-0.021033,-0.008908,-0.022090,-0.021032,-0.065256,0.020567,-0.032405,0.036942,0.841666,0.999457,0.949299,0.522119,0.068154,0.020298,-0.003951,0.013926,0.004439,NaN,NaN,NaN,0.006775,-0.032405,-0.043746,0.004835,0.007375,0.009191,0.002252,0.042755,0.025105,0.022799,0.028002,0.006007,0.005759,0.001839,0.004587,0.003962,0.997972,0.998911,0.999465,0.047587,0.997952,0.998901,0.999458,0.047582,0.016532,0.027610,0.029712,0.003785,0.004838,0.082955,0.067066
total_fpackets,0.004837,1.000000,0.924622,0.425756,0.904007,-0.018958,0.005252,0.024685,0.086255,-0.007910,0.139142,0.010172,0.020324,0.002190,0.016479,-0.001975,-0.000651,-0.000420,0.008979,-0.001970,-0.000694,0.000037,0.000619,0.175476,0.174778,NaN,NaN,0.999838,0.924622,-0.004898,-0.002443,-0.005194,-0.003356,-0.019531,0.074042,0.098944,0.087741,-0.001972,0.002273,-0.002050,-0.000211,0.005461,0.045730,-0.004097,0.184351,0.965916,NaN,NaN,NaN,0.249932,0.098944,-0.007910,0.999838,0.040468,0.073107,0.001703,0.012731,0.139151,0.163392,0.143763,0.030887,0.859868,0.350716,0.812197,0.794283,0.002195,0.002580,0.003261,0.018109,0.001258,0.001614,0.002267,0.017229,0.016089,0.050201,0.059224,0.902713,0.999866,0.018198,0.018377
total_bpackets,0.004011,0.924622,1.000000,0.156780,0.997268,-0.017667,0.006912,0.018170,0.086886,-0.016104,0.151761,0.002331,0.014005,0.001718,0.014774,-0.002172,-0.000542,-0.000714,0.007659,-0.002241,-0.000797,-0.000390,-0.000354,0.136986,0.207102,NaN,NaN,0.924199,1.000000,-0.005400,-0.001277,-0.005533,-0.003341,-0.017999,0.073497,0.109240,0.087307,-0.002169,0.001623,-0.002292,-0.000653,0.004894,0.043305,-0.004343,0.201262,0.991675,NaN,NaN,NaN,0.306614,0.109240,-0.016104,0.924199,0.022299,0.059453,0.000605,0.006942,0.151778,0.183837,0.156579,0.033440,0.745095,0.124747,0.806157,0.804088,0.001495,0.001852,0.002592,0.017605,0.000610,0.000922,0.001617,0.016230,-0.000493,0.048190,0.058435,0.997580,0.924746,0.015124,0.019430
total_fpktl,0.001673,0.425756,0.156780,1.000000,0.090082,-0.003099,0.000803,0.021278,0.022088,0.022409,0.018954,0.011416,0.007763,0.000708,0.006442,-0.000746,-0.000185,-0.000389,0.003309,-0.000728,-0.000025,0.000059,0.000960,0.197453,0.055171,NaN,NaN,0.425588,0.156780,-0.001868,-0.000810,-0.001964,-0.001427,-0.003763,0.022072,0.027087,0.024298,-0.000744,0.000611,-0.000748,0.000119,-0.001464,0.012057,-0.001550,0.092604,0.250470,NaN,NaN,NaN,0.026329,0.027087,0.022409,0.425588,0.055662,0.015117,0.001867,0.004947,0.018949,0.020708,0.027528,0.004007,0.342773,0.819266,0.132894,0.074529,0.000434,0.000650,0.000919,0.009627,0.000113,0.000335,0.000609,0.009896,0.001657,0.013283,0.015991,0.088422,0.425789,0.005477,0.000679
total_bpktl,0.003518,0.904007,0.997268,0.090082,1.000000,-0.014926,0.005966,0.012560,0.079905,-0.017328,0.146437,-0.003

In [28]:
# Se puede llegar a valorar quedarse con aquellas con mayor correlacion
corr_matrix[corr_matrix['calss']>0.05]

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward,calss
duration,1.000000,0.004837,0.004011,0.001673,0.003518,-0.064100,-0.027231,0.008761,0.042925,-0.043746,0.025117,0.039350,0.048743,0.943898,0.324705,0.841692,0.212482,0.943438,0.324226,0.918036,0.280637,0.421540,0.237307,0.021372,0.010780,NaN,NaN,0.004835,0.004011,-0.021033,-0.008908,-0.022090,-0.021032,-0.065256,0.020567,-0.032405,0.036942,0.841666,0.999457,0.949299,0.522119,0.068154,0.020298,-0.003951,0.013926,0.004439,NaN,NaN,NaN,0.006775,-0.032405,-0.043746,0.004835,0.007375,0.009191,0.002252,0.042755,0.025105,0.022799,0.028002,0.006007,0.005759,0.001839,0.004587,0.003962,0.997972,0.998911,0.999465,0.047587,0.997952,0.998901,0.999458,0.047582,0.016532,0.027610,0.029712,0.003785,0.004838,0.082955,0.067066
max_bpktl,0.042925,0.086255,0.086886,0.022088,0.079905,-0.277317,0.275923,0.492194,1.000000,-0.018358,0.895712,0.564243,0.941626,-0.005961,0.174120,-0.035963,-0.003390,-0.009878,0.162721,-0.032412,0.031298,0.021591,0.137169,0.294760,0.167710,NaN,NaN,0.086204,0.086886,-0.089801,-0.014859,-0.091152,-0.081798,-0.282852,0.851572,0.387270,0.911584,-0.035917,0.038796,-0.031180,0.042461,0.035163,0.537187,-0.040593,0.207509,0.087864,NaN,NaN,NaN,0.443414,0.387270,-0.018358,0.086204,0.162106,0.254060,0.064046,0.900543,0.895560,0.557349,0.653017,0.189258,0.056164,0.015515,0.056596,0.052320,0.033057,0.035932,0.039286,0.128193,0.032544,0.035413,0.038732,0.127548,-0.044916,0.586742,0.593143,0.084280,0.086278,0.217989,0.073212
mean_bpktl,0.025117,0.139142,0.151761,0.018954,0.146437,-0.280648,0.465208,0.342392,0.895712,-0.096195,1.000000,0.406878,0.744870,-0.009298,0.127441,-0.036990,-0.001386,-0.012626,0.117203,-0.034037,0.018819,0.017698,0.085891,0.253869,0.176532,NaN,NaN,0.139067,0.151761,-0.092269,-0.007038,-0.092535,-0.081023,-0.285849,0.728284,0.431783,0.867507,-0.036954,0.021587,-0.033885,0.027641,-0.018607,0.413631,-0.048392,0.204764,0.149739,NaN,NaN,NaN,0.648624,0.431783,-0.096195,0.139067,0.129846,0.288076,0.046447,0.710476,1.000000,0.714473,0.742566,0.232429,0.090559,0.011901,0.099357,0.096353,0.016410,0.019000,0.022020,0.115800,0.015961,0.018533,0.021519,0.114949,-0.045442,0.452735,0.478823,0.150252,0.139172,0.122226,0.064753
std_fpktl,0.039350,0.010172,0.002331,0.011416,-0.003162,-0.245792,0.052877,0.817873,0.564243,0.259588,0.406878,1.000000,0.598272,-0.003338,0.164844,-0.030462,0.000778,-0.008382,0.149920,-0.025088,0.041391,0.026499,0.143370,0.260774,0.094071,NaN,NaN,0.010157,0.002331,-0.065727,-0.033032,-0.069738,-0.048775,-0.252067,0.706382,0.247325,0.738951,-0.030449,0.034024,-0.024454,0.045162,0.108340,0.508487,-0.054598,0.139512,0.004901,NaN,NaN,NaN,0.043725,0.247325,0.259588,0.010157,0.219957,0.105756,0.082116,0.460822,0.406530,0.161492,0.260807,0.053297,0.005395,0.007480,0.000445,-0.002807,0.029585,0.031783,0.034376,0.097515,0.029228,0.031418,0.033978,0.096844,-0.020885,0.558249,0.532900,0.000140,0.010182,0.301237,0.123758
std_bpktl,0.048743,0.020324,0.014005,0.007763,0.007768,-0.225143,0.035371,0.534532,0.941626,0.051756,0.744870,0.598272,1.000000,0.0

### 3.- Division del DataSet

In [29]:
# Division del DataSet
train_set, val_set, test_set = train_val_test_split(X)

In [30]:
X_train, y_train = remove_labels(train_set, 'calss')
X_val, y_val = remove_labels(val_set, 'calss')
X_test, y_test = remove_labels(test_set, 'calss')


### 4.- Escalado del DataSet

Es importante comprender que los arboles de decision son algoritmos que **no requieren demaciada preparacion de los datos** concretamente, no requiere la realizacion o escalado o normalizacion. En este ejercisio se va a realizar escalado al DataSet y se van a comparr los resultados con el DataSet sin escalar. De esdta manera se muestra como aplicar preprocesamientos de como el escalado puede llegar a afectar al rendimiento del modelo 

In [31]:
scaler = RobustScaler()
X_train_scaler = scaler.fit_transform(X_train)

In [32]:
scaler = RobustScaler()
X_test_scaler = scaler.fit_transform(X_test)

In [33]:
scaler = RobustScaler()
X_val_scaler = scaler.fit_transform(X_val)

In [35]:
# Trasformacion de los datos
from pandas import DataFrame

X_train_scaled = DataFrame(X_train_scaler, columns = X_train.columns, index = X_train.index)
X_train_scaled.head(10)

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward
508881,-0.013646,0.0,1.0,-0.310056,1.556886,0.375000,4.924528,-0.027100,2.250000,-0.032895,2.494005,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,68.333145,127.926531,67.693073,110.512958,0.375000,0.267568,0.186901,1.449936,0.028810,-0.018341,-0.020676,0.00000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.0,0.0,3.561644,0.186901,-0.032895,0.0,0.0,0.0,0.0,0.00000,2.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.286342,0.000011,0.0,0.0,-1.0
208326,-0.013926,0.0,0.0,0.664804,0.000000,6.607143,0.000000,0.918699,0.000000,1.115132,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008692,0.000000,-0.006927,-0.008810,6.607143,0.705405,1.003195,-0.011049,-0.000237,-0.018729,-0.021737,0.00000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.0,0.0,0.000000,1.003195,1.115132,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.286342,0.000000,0.0,0.0,-1.0
107213,-0.013926,0.0,0.0,0.703911,0.000000,6.857143,0.000000,0.956640,0.000000,1.161184,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008692,0.000000,-0.006927,-0.008810,6.857143,0.743243,1.047923,-0.011049,-0.000237,-0.018729,-0.021737,0.00000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.0,0.0,0.000000,1.047923,1.161184,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.286342,0.000000,0.0,0.0,-1.0
466726,-0.000273,0.0,1.0,-0.363128,2.724551,0.035714,8.603774,-0.078591,3.931034,-0.095395,4.364508,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.390527,2.619145,1.379151,3.449949,0.035714,0.794595,0.468051,3.121935,1.415641,0.000180,0.030094,0.00000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.0,0.0,8.425926,0.468051,-0.095395,0.0,0.0,0.0,0.0,0.00000,4.375000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.286342,0.000011,0.0,0.0,-1.0
230085,-0.013926,0.0,0.0,0.664804,0.000000,6.607143,0.000000,0.918699,0.000000,1.115132,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008692,0.000000,-0.006927,-0.008810,6.607143,0.705405,1.003195,-0.011049,-0.000237,-0.018729,-0.021737,0.00000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.0,0.0,0.000000,1.003195,1.115132,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.286342,0.000000,0.0,0.0,-1.0
472961,34.421927,1.5,4.0,1.558659,3.868263,0.000000,1.000000,1.341463,4.163793,0.337171,1.549161,16.895157,213.699165,0.336088,60331999.0,1.062857e+03,11262.0,0.289293,60228809.0,0.467228,20100000.0,2.777741,34700000.0,1.0,1.0,0.0,0.0,1.5,4.0,-0.006473,0.004154,-

In [36]:
X_train_scaled.describe()

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,mean_bpktl,std_fpktl,std_bpktl,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,std_fiat,std_biat,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,fPktsPerSecond,bPktsPerSecond,flowPktsPerSecond,flowBytesPerSecond,min_flowpktl,max_flowpktl,mean_flowpktl,std_flowpktl,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,flow_fin,flow_syn,flow_rst,flow_psh,flow_ack,flow_urg,flow_cwr,flow_ece,downUpRatio,avgPacketSize,fAvgSegmentSize,fHeaderBytes,fAvgBytesPerBulk,fAvgPacketsPerBulk,fAvgBulkRate,bVarianceDataBytes,bAvgSegmentSize,bAvgBytesPerBulk,bAvgPacketsPerBulk,bAvgBulkRate,sflow_fpacket,sflow_fbytes,sflow_bpacket,sflow_bbytes,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward
count,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,3.791730e+05,3.791730e+05,3.791730e+05,379173.000000,3.791730e+05,379173.000000,3.791730e+05,3.791730e+05,3.791730e+05,379173.000000,379173.000000,379173.0,379173.0,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.0,379173.0,379173.0,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,3.791730e+05,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000,379173.000000
mean,12.543409,2.927643,10.762678,1.947184,75.104138,1.600682,0.855241,0.491724,1.591823,0.303639,1.014557,3.439516,61.636887,38.860997,7.147523e+06,8.335603e+05,9.511070e+05,58.145453,6.298974e+06,126.177236,2.063700e+06,1.641220e+02,2.243039e+06,0.861778,1.070132,0.0,0.0,2.931056,10.762678,984.393314,110.028225,516.685557,269.838608,1.583599,0.488420,0.299527,0.929250,6.885496e+02,16.453111,28.842890,1.870475e+02,0.529429,0.429477,0.066421,1.931910,4.956526,0.0,0.0,0.0,1.243744,0.299527,0.303639,2.931056,53.583019,0.336585,2.285299e+05,28454.905968,1.016042,457.111767,0.494429,2.143836e+05,3.197622,7.910795,5.196285,6.398888e+03,19.698746,14.179793,14.417010,4.669377e+05,19.838596,16.279309,16.441899,4.686618e+05,0.362658,2.861506,3.435883,10.087709,2.924548,-0.376798
std,116.165117,92.319402,370.875546,185.549801,3071.968461,2.817301,1.676098,0.786500,3.210186,0.533388,1.985607,7.959015,157.021881,453.629167,7.145611e+07,1.171375e+07,4.433420e+07,724.373302,7.109207e+07,1690.470033,4.804956e+07,6.204794e+03,3.133505e+07,3.804841,13.671129,0.0,0.0,92.381891,370.875546,5433.357640,1419.815251,2712.294228,1443.505060,2.738992,1.038379,0.551685,1.764197,9.700870e+03,160.819659,374.179837,4.308946e+03,0.652334,0.911635,0.302768,16.541754,182.595272,0.0,0.0,0.0,4.791424,0.551685,0.533388,92.381891,604.670250,4.022938,6.486112e+06,88196.057761,1.989369,2552.776459,2.156685,3.469284e+06,162.611348,1271.541381,295.218736,4.090311e+05,199.389786,141.185645,140.610872,6.181972e+06,201.477623,162.562856,160.822696,6.188642e+06,3.836035,5.577212,7.354164,369.660712,92.301146,0.466297
min,-0.013936,-0.500000,0.000000,-0.513966,0.000000,-0.946429,0.000000,-0.227642,0.000000,-0.273026,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,-0.500000,0.000000,-0.008692,0.000000,-0.006927,-0.008810,-0.339286,-0.3459

### 5.- Decision Forest

In [37]:
# Modelo entrenado con el DataSet sin escalar

from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier(random_state=42)
clf_tree.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None
